In [2]:
import pandas as pd

In [51]:
# Import and concatenate all datasets

df1 = pd.read_csv("2019 - 01.csv")
df2 = pd.read_csv("2019 - 02.csv")
df3 = pd.read_csv("2019 - 03.csv")
df4 = pd.read_csv("2019 - 04.csv")
df = pd.read_csv("2019 - 05.csv")
df5 = pd.concat([df1, df2, df3, df4, df5])

df["Target ID"] = df["end_station_name"]
df["Source ID"] = df["start_station_name"]
df["Name"] = df["start_station_name"]

# Save the new file to your downloads
#df.to_csv("data_lyft.csv")

In [52]:
df1 = df.groupby(['start_station_name','end_station_name']).size().reset_index().rename(columns={0:'count'})
df1["Target ID"] = df["end_station_name"]
df1["Source ID"] = df["start_station_name"]
df1["Name"] = df["start_station_name"]
df1

,start_station_name,end_station_name,count,Target ID,Source ID,Name
0,10th Ave at E 15th St,10th St at Fallon St,20,8th St at Ringold St,5th St at Folsom,5th St at Folsom
1,10th Ave at E 15th St,2nd Ave at E 18th St,1,Parker St at Fulton St,Berkeley Civic Center,Berkeley Civic Center
2,10th Ave at E 15th St,32nd St at Adeline St,2,Emeryville Town Hall,Emeryville Town Hall,Emeryville Town Hall
3,10th Ave at E 15th St,4th Ave at E 12th St (Temporary Location),1,Lakeside Dr at 14th St,Lakeside Dr at 14th St,Lakeside Dr at 14th St
4,10th Ave at E 15th St,6th Ave at E 12th St (Temporary Location),1,Myrtle St at Polk St,Father Alfred E Boeddeker Park,Father Alfred E Boeddeker Park
...,...,...,...,...,...,...
23507,Yerba Buena Center for the Arts (Howard St at ...,Victoria Manalo Draves Park,20,San Francisco Caltrain Station 2 (Townsend St...,Yerba Buena Center for the Arts (Howard St at ...,Yerba Buena Center for the Arts (Howard St at ...
23508,Yerba Buena Center for the Arts (Howard St at ...,Washington St at Kearny St,7,Steuart St at Market St,The Embarcadero at Sansome St,The Embarcadero at Sansome St
23509,Yerba Buena Center for the Arts (Howard St at ...,Webster St at Grove St,4,Alcatraz Ave at Shattuck Ave,Ashby BART Station,Ashby BART Station
23510,Yerba Buena Center for the Arts (Howard St at ...,Webster St at O'Farrell St,3,San Francisco Caltrain Station 2 (Townsend St...,Montgomery St BART Station (Market St at 2nd St),Montgomery St BART Station (Market St at 2nd St)


In [59]:
# Prerequisites

import sys
!{sys.executable} -m pip install --user networkx
!{sys.executable} -m pip install --user numpy
!{sys.executable} -m pip install --user pandas
import networkx as nx
import numpy as np
import pandas as pd
from pandas import DataFrame

# Set source and target column

df_InputData = df1
Src_Column = 'Source ID'
Tgt_Column = 'Target ID'

In [60]:
# Nodes are positioned using the Fruchterman-Reingold force-directed algorithm

Q = nx.Graph()
arr_SrcTgt= np.array(df_InputData[[Src_Column, Tgt_Column]])
Q.add_edges_from(arr_SrcTgt)
dict_Coords = nx.spring_layout(Q)

df_Raw_Coords = DataFrame(dict_Coords)
df_Raw_Coords = df_Raw_Coords.T
df_Raw_Coords.columns = ['X','Y']
df_Raw_Coords['NodeName'] = df_Raw_Coords.index

# Just in case some nodes to not link to another

df_Raw_Coords.fillna("Not Specified", inplace = True)

In [61]:
# Create bridge between main dataset and coordinates

arr_SrcTgt2 = arr_SrcTgt.reshape(1,(len(arr_SrcTgt)*2))
arr_SrcTgt2 = arr_SrcTgt2.reshape(-1)
df_SrcTgt = DataFrame(arr_SrcTgt2,columns=['NodeName'])
arr_Index = []
for i in range(1,(len(arr_SrcTgt)+1)):
        arr_Index.append(i)
        arr_Index.append(i)
df_SrcTgt['c_Index'] = arr_Index

# Join the datasets

df_InputData.index = df_InputData.index + 1

Merge_1 = pd.merge(
    left=df_SrcTgt,
    right=df_InputData,
    how="inner",
    left_on=df_SrcTgt['c_Index'],
    right_index=True,
)

df_MainDat = DataFrame(Merge_1)
df_MainDat = df_MainDat.drop(columns=['key_0'])

Merge_2 = pd.merge(
    left=df_Raw_Coords,
    right=df_MainDat,
    how="left",
    left_on=df_Raw_Coords['NodeName'],
    right_on=df_MainDat['NodeName'],
    suffixes=("", "_y"),
)

df_finaldat = DataFrame(Merge_2)
df_finaldat = df_finaldat.drop(columns=['key_0','NodeName_y'])

In [62]:
# Save the dataset

df_finaldat.to_csv("Lyft Data_v06.csv")

In [63]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23512 entries, 1 to 23512
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   start_station_name  23512 non-null  object
 1   end_station_name    23512 non-null  object
 2   count               23512 non-null  int64 
 3   Target ID           23512 non-null  object
 4   Source ID           23512 non-null  object
 5   Name                23512 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.1+ MB
